In [1]:
# Spliting dataset into train, valuation and test sets.
!pip install split_folders
import splitfolders
input_folder = 'data'
output = 'processed_data'
splitfolders.ratio(input_folder, output, seed=42, ratio=(.6, .3, .1))

Copying files: 717 files [00:08, 88.63 files/s] 


In [1]:
!pip install tensorflow opencv-python matplotlib pandas scipy seaborn scikit-learn

^C


You should consider upgrading via the 'C:\Users\abhay\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
# Importing libraries 
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.losses import CategoricalCrossentropy

In [3]:
# Creating variables to access the processed data sets.
train_dir = 'processed_data/train' 
val_dir = 'processed_data/val'
test_dir = 'processed_data/test'

In [4]:
#Generting the training datasets
datagen = ImageDataGenerator(preprocessing_function=preprocess_input, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

train_generator = datagen.flow_from_directory(train_dir, target_size=(256, 256), batch_size=32, class_mode='categorical')
val_generator = datagen.flow_from_directory(val_dir, target_size=(256, 256), batch_size=32, class_mode='categorical')
test_generator = datagen.flow_from_directory(test_dir, target_size=(256, 256), batch_size=1, class_mode='categorical')

Found 567 images belonging to 7 classes.
Found 325 images belonging to 7 classes.
Found 135 images belonging to 7 classes.


In [5]:
# Creating training model based on Residual Neural Network (ResNet) and predicition layers created categorically for the 10 categories.
# Model 1 - ResNet50
base_model = ResNet50(include_top=False, weights="imagenet")

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
output = Dense(train_generator.num_classes, activation="softmax")(x)

final_model = Model(inputs=base_model.input, outputs=output)
final_model.compile(optimizer="adam", loss=CategoricalCrossentropy(reduction='sum_over_batch_size'), metrics=["accuracy"])
final_model.fit(train_generator, epochs=50, validation_data=val_generator)
final_model.save('saved_models/artportal_model_v1_03.h5')

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 10s 0us/step


c:\Users\abhay\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 141s 8s/step - accuracy: 0.4365 - loss: 2.1546 - val_accuracy: 0.7385 - val_loss: 0.8161
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 137s 8s/step - accuracy: 0.7755 - loss: 0.6110 - val_accuracy: 0.8246 - val_loss: 0.5200
Epoch 3/50
17/18 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.8801 - loss: 0.3070

KeyboardInterrupt: 

In [ ]:
model = tf.keras.models.load_model('saved_models/artportal_model_v1_01.h5')
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print('\n Test accuracy:', test_acc)

135/135 - 29s - loss: 0.4313 - accuracy: 0.9111 - 29s/epoch - 218ms/step

 Test accuracy: 0.9111111164093018
